# LASER LOCK

In [ ]:
# import libraries for devices
import time, datetime

# import libraries for plotting etc
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## 1. Connect to the RedPitaya

- The config file should already have all the modules opened up, otherwise open them and set all the values accordingly ( see labNotes...)
- Start ramping with the full range of 1 V such that you do not miss the signal

In [ ]:
import pyrpl

In [ ]:
p = pyrpl.Pyrpl(hostname="10.0.2.111", config="111.yml")

## 2. Connect to the Moku:GO
- It may be, that the moku is already inizialized and won't respond. In that case, clear all connections first and then retry

In [ ]:
from moku.instruments import ArbitraryWaveformGenerator, Oscilloscope

In [ ]:
# Close the connection to the Moku device
# This ensures network resources and released correctly
# i.relinquish_ownership()
    
i = ArbitraryWaveformGenerator('10.0.2.112', force_connect=True)

## 3. Find a good offset for the ramping

In [ ]:
# Here you can access the available_power_supplies function
#i.available_power_supplies()

# We usually have to set the ramping off by about 2 V...
i.set_power_supply(1, enable=True, voltage=0, current=0.0)

## 4. Zoom in unto the CO-resonance
- Decrease the amplitude of the ramping while adjusting the offset given by the Moku:GO such that the resonance stays in the center of the slope

## 5. Actual locking
- Turn on the fast ( 10 kHz ) modulation via the coil. Check for the iq output
- Set the setpoint of the PID, turn of the ramp and forward the PID output to the laser ( out 1 )

Enjoy your lock!

# APPENDIX

## Moku:GO Oscilloscope

In [ ]:
# Here we want to have the plots in an external window
%matplotlib

In [ ]:
i.relinquish_ownership()
# i = Oscilloscope('10.0.2.112', force_connect=False)

In [ ]:
i = Oscilloscope('10.0.2.112', force_connect=True)

In [ ]:
# This example demonstrates how you can configure the Oscilloscope instrument,
# and view triggered time-voltage data frames in real-time.
#
# (c) 2021 Liquid Instruments Pty. Ltd.
def on_close(event):
    print('Closed Figure!')
    i.relinquish_ownership()

try:
    # Trigger on input Channel 1, rising edge, 0V 
    i.set_trigger(type='Edge', source='Input1', level=0)

    # View +-5usec, i.e. trigger in the centre
    i.set_timebase(-5e-6, 5e-6)

    # Set the data source of Channel 1 to be Input 1
    i.set_source(1, 'Input1')

    # Set the data source of Channel 2 to the generated output sinewave
    i.set_source(2, 'Input2')


    # Get initial data frame to set up plotting parameters. This can be done
    # once if we know that the axes aren't going to change (otherwise we'd do
    # this in the loop)
    data = i.get_data()

    # Set up the plotting parameters
    plt.ion()
    plt.show()
    plt.grid(b=True)
    plt.ylim([-1, 1])
    plt.xlim([data['time'][0], data['time'][-1]])

    line1, = plt.plot([], label='channel 1')
    line2, = plt.plot([], label='channel 2')

    # Configure labels for axes
    ax = plt.gca()
    
    # This loops continuously updates the plot with new data
    while True:
        # Get new data
        data = i.get_data()

        # Update the plot
        line1.set_ydata(data['ch1'])
        line2.set_ydata(data['ch2'])
        line1.set_xdata(data['time'])
        line2.set_xdata(data['time'])
        
        ax.set_title(datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S"))
        ax.legend()

        plt.pause(0.001)
        
        fig = plt.gcf()  # get current figure
        fig.canvas.mpl_connect('close_event', on_close)
        
except Exception as e:
    print(f'Exception occurred: {e}')
except KeyboardInterrupt:
    print('Interrupt!')
finally:
    # Close the connection to the Moku device
    # This ensures network resources and released correctly
    i.relinquish_ownership()
    print('Ownership relinquished!')